In [ ]:
import pandas as pd
import pymc3 as pm
import numpy as np
import theano
import theano.tensor as tt


In [ ]:
data = pd.read_csv("HerrmannThoeniGaechterDATA.csv", skiprows=3)

In [ ]:
len(data)

In [ ]:
data.head()

In [ ]:
## constants
groupSize = 4
ntrials = 10
pi = 1.4
ntokens = 20
vals = list(range(21)) #possible values to contribute - from 0 to 20 tokens

## Prelim checks

In [ ]:
len(set(data["subjectid"])) #should be 1120

In [ ]:
len(set(data["groupid"]))

In [ ]:
data.dtypes

In [ ]:
def get_gini(city):
    if city == "Melbourne":
        return 34.3
    if city == "Minsk":
        return 25.3
    if city == "Chengdu":
        return 38.5
    if city == "Copenhagen":
        return 28.7
    if city == "Bonn":
        return 31.9
    if city == "Athens":
        return 34.4
    if city == "Seoul":
        return 31.6
    if city == "Samara":
        return 37.5
    if city == "Zurich":
        return 32.7
    if city == "St. Gallen":
        return 32.7
    if city == "Istanbul":
        return 41.9
    if city == "Nottingham":
        return 34.8
    if city == "Dnipropetrovs'k":
        return 26.1
    if city == "Boston":
        return 41.1
    
data["gini"] = data["city"].apply(lambda x: get_gini(x))

In [ ]:
#data = data[data["gini"].isna() == False]
#data = data.dropna()

In [ ]:
len(data)

In [ ]:
redDat = data.iloc[::3, :] #get only every third row, the others are other responses that we don't need

In [ ]:
redDat.reset_index(inplace = True, drop = True)

In [ ]:
group_names = set(redDat["groupid"])
group_names = sorted(list(group_names))
ngroups = len(group_names)

### DOES SOMETHING FUCKED HERE

subject_names = set(redDat["subjectid"])
nsubjects = len(subject_names)
ngroups = 269

In [ ]:
## No punish initialized
c_no_punish = np.zeros(shape = (groupSize, ntrials, ngroups))
Ga_no_punish = np.zeros(shape = (ntrials, ngroups))
Gc_no_punish = np.zeros(shape = (groupSize, ntrials, ngroups))

#punished initialized
c_punish = np.zeros(shape = (groupSize, ntrials, ngroups))
Ga_punish = np.zeros(shape = (ntrials, ngroups))
Gc_punish = np.zeros(shape = (groupSize, ntrials, ngroups))

#missing vector
missing = np.zeros(shape = (ngroups))

In [ ]:
for g in range(ngroups):
    ### fancy way of fixing missing data - they will just become 0, 
    # and their position is then logged in "missing"-vector
    try: 
        #no punish
        c_no_punish[:,:,g][0] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "N-experiment")]["senderscontribution"][0:10].values
        c_no_punish[:,:,g][1] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "N-experiment")]["senderscontribution"][10:20].values
        c_no_punish[:,:,g][2] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "N-experiment")]["senderscontribution"][20:30].values
        c_no_punish[:,:,g][3] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "N-experiment")]["senderscontribution"][30:40].values
        
        #punish
        c_punish[:,:,g][0] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "P-experiment")]["senderscontribution"][0:10].values
        c_punish[:,:,g][1] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "P-experiment")]["senderscontribution"][10:20].values
        c_punish[:,:,g][2] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "P-experiment")]["senderscontribution"][20:30].values
        c_punish[:,:,g][3] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "P-experiment")]["senderscontribution"][30:40].values
    
    except: #could be more general... 
        missing[g] = 1
    
    #make "Ga"
    
    Ga_no_punish[:,g] = c_no_punish[:,:,g].mean(axis=0)
    Ga_punish[:,g] = c_punish[:,:,g].mean(axis=0)
    
    #make "Gc"
    for s in range(groupSize):
        Gc_no_punish[:,:,g][s] = np.delete(c_no_punish[:,:,g], s, 0).sum(axis=0) #just gives a single number though...
        Gc_punish[:,:,g][s] = np.delete(c_punish[:,:,g], s, 0).sum(axis=0)

### Concat the different conditions

In [ ]:
c = np.zeros(shape = (groupSize, ntrials, ngroups, 2))
Ga = np.zeros(shape = (ntrials, ngroups, 2))

In [ ]:
c.shape

In [ ]:
c[:,:,:,0] = c_no_punish
c[:,:,:,1] = c_punish

Ga[:,:,0] = Ga_no_punish
Ga[:,:,1] = Ga_punish

### Gini Coefficient

In [ ]:
Gini = np.zeros(shape = ngroups)
for g in range(ngroups):
    Gini[g] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "P-experiment")]["gini"].values.mean()

In [ ]:
mask = np.isnan(Gini) == False

In [ ]:
#Ga_punish = Ga_punish[:,mask]
#Ga_no_punish = Ga_no_punish[:,mask]

c = c[:,:,mask,:]
Ga = Ga[:,mask,:]
#Gc = Gc[:,:,mask,:]
Gini = Gini[mask]

#get new ngroups
ngroups = len(Gini)

### Prelim check

First we need to add all contributions in a group and multiply them by 1.4 and calculate the Pearson correlation between Gini coefficients and the earnings.


### Heat map

Lower Quartile (Gini < 31.6)
Upper Quartile (Gini > 37.5)

## Decay model

$$c_{g,s,t}^\mu = c^0_{g,s} \cdot e^{-\gamma_{s,g} \cdot t}$$

$$c^0_{g,s} = \beta^c_0 + \beta^c_{Gini} \cdot Gini_g$$

$$\gamma_{s,g} = \beta^\gamma_0 + \beta^\gamma_{Gini} \cdot Gini_g$$

In [ ]:
y = c[:,:,:,0]

In [ ]:
idx_g = np.arange(ngroups)
idx_s = np.array([0,1,2,3])
t = np.arange(ntrials)

In [ ]:
t_stack = np.stack((t,t,t,t))
t_stack = t_stack.T


In [85]:
t_stack

array([[0, 0, 0, 0],
       [1, 1, 1, 1],
       [2, 2, 2, 2],
       [3, 3, 3, 3],
       [4, 4, 4, 4],
       [5, 5, 5, 5],
       [6, 6, 6, 6],
       [7, 7, 7, 7],
       [8, 8, 8, 8],
       [9, 9, 9, 9]])

In [ ]:
Gini_stack = np.stack((Gini, Gini, Gini, Gini))
Gini_stack = Gini_stack.T

In [ ]:
y.shape

In [ ]:
Gini_stack.shape

In [ ]:
idx_s = idx_s[:, None]
#idx_g = idx_g[:,None]

In [ ]:
idx_s

In [ ]:
idx_g

In [ ]:
Gini_stack[idx_g, idx_s]

In [ ]:
Gini_stack

In [ ]:
with pm.Model() as m:
    ## priors
    beta0_c0 = pm.HalfNormal("beta0_c0", 1)
    betaGini_c0 = pm.Normal("betaGini_c0", 0, 1)
    
    beta0_gamma = pm.HalfNormal("beta0_gamma", 1)
    betaGini_gamma = pm.Normal("betaGini_gamma", 0, 1)
    
    ## group_level
    sigma_c = pm.Gamma("sigma_c", 1, 1, shape = (ngroups, groupSize))
    
    c_0 = pm.Deterministic("c_0", beta0_c0 + betaGini_c0 * Gini_stack[idx_g, idx_s])
    
    gamma = pm.Deterministic("gamma", beta0_gamma + betaGini_gamma * Gini_stack[idx_g, idx_s])
    
    ### with time 
    
    tester = pm.math.dot(t_stack, -gamma)
    
    #test_2 = pm.math.dot(tester, c_0)
    
    mu_c = pm.Deterministic("mu_c", 
                            pm.math.dot(c_0, pm.math.exp(pm.math.dot(t_stack, -gamma)))) ## this is gonna be fucked
    
    ## likelihood
    
    y_pred = pm.Normal("y_pred", mu = mu_c, sigma = sigma_c, observed = y)
    
    

244

In [88]:
with pm.Model() as m:
    ## priors
    beta0_c0 = pm.HalfNormal("beta0_c0", 1)
    betaGini_c0 = pm.Normal("betaGini_c0", 0, 1)
    
    beta0_gamma = pm.HalfNormal("beta0_gamma", 1)
    betaGini_gamma = pm.Normal("betaGini_gamma", 0, 1)
    
    ## group_level
    sigma_c = pm.Gamma("sigma_c", 1, 1, shape = (ngroups, groupSize)) 
    
    c_0 = pm.Deterministic("c_0", beta0_c0 + betaGini_c0 * Gini_stack[idx_g, idx_s])
    
    gamma = pm.Deterministic("gamma", beta0_gamma + betaGini_gamma * Gini_stack[idx_g, idx_s])
    
    ### with time 
    
    mu_c = pm.Deterministic("mu_c", pm.math.dot(c_0, pm.math.exp(pm.math.dot(t_stack, -gamma)))) ## this is gonna be fucked
    
    ## likelihood
    
    y_pred = pm.Normal("y_pred", mu = mu_c, sigma = sigma_c, observed = y)
    
    

### Preferences:

$$pvec = p_0 + \beta_p \cdot tokenvalues $$

$$p_t = pvec[Gb_t]$$

where Gb_t is the belief of what the other person will do at time T

$$Gb_t = \gamma ( Gb_{t-1}) + (1-\gamma) (G_{a_{t-1}})$$

$$c_t = \omega_t (Gb_t) + (1-\omega_t) (p_t)$$

$$\omega_t = \omega_{t-1}(1-\lambda)$$

### Getting the hierachical model

$$B^P_{g,s} \sim Beta(Shape_{g,s,1}, Shape_{g,s,2})$$

$$Shape_1 = \mu B^P_{g,s} + \sigma_{g,s}$$
$$Shape_2 = (1 - \mu B^P_{g,s}) + \sigma_{g,s}$$

$$Probit(\mu B^P_{g,s}) = \beta_0^P + \beta^P_{Gini} \cdot Gini$$